<a href="https://colab.research.google.com/github/towardsai/cours-demo-notebooks/blob/main/exercice_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# Installation des bibliothèques nécessaires (seulement pour Colab)
# !pip install instructor pydantic google-generativeai jsonref

In [1]:
# Importation des bibliothèques
import instructor
import google.generativeai as genai
from pydantic import BaseModel, Field
from enum import Enum
from typing import Literal
from dotenv import load_dotenv
import os

/Users/omar/Documents/ai_repos/cours-demo-notebooks/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuration de l'API Gemini

load_dotenv(".env")

GOOGLE_API_KEY = ""  # Remplacez par votre clé API
genai.configure(api_key=os.environ["GOOGLE_AI_STUDIO"])

In [3]:
# 1. Définition des modèles Pydantic
class Personne(BaseModel):
    nom: str = Field(max_length=50)
    age: int | str
    role: str = Field(
        description="Role de la personne dans la compagnie, ex: Développeur",
    )
    compagnie: Literal["NeuroTech Systems", "VirtuaSoft", "Quantum Solutions", "OTHER"]


class ExtractionPersonnes(BaseModel):
    reasoning: str = Field(
        description="À partir des informations fournies, écris 'comment' tu vas t'y prendre pour extraire les informations du texte.",
    )
    list_personnes: list[Personne] = Field(min_length=3)

In [4]:
# 2. Initialisation du client Gemini avec JSON STRICT pour l'extraction SEULEMENT
client = instructor.from_gemini(
    client=genai.GenerativeModel(model_name="models/gemini-2.0-flash"),
    mode=instructor.Mode.GEMINI_JSON,  # Forcer JSON uniquement pour l'extraction
)

In [5]:
# 3. Texte d'entrée
texte = """
Thomas, 34 ans, est ingénieur en intelligence artificielle chez NeuroTech Systems, une startup spécialisée dans les algorithmes d’apprentissage profond.
Il passe ses journées à entraîner des modèles pour optimiser la reconnaissance vocale, souvent en collaboration avec Léa, 29 ans, designer UX chez VirtuaSoft,
une entreprise qui développe des interfaces immersives pour la réalité augmentée.
Léa s’assure que les utilisateurs interagissent intuitivement avec les IA conçues par des entreprises comme celle de Thomas.
De son côté, Karim, 41 ans, est chef de projet chez Quantum Solutions, un cabinet de conseil en innovation technologique.
Il supervise des partenariats entre sociétés comme NeuroTech et VirtuaSoft, coordonnant leurs efforts pour créer des expériences utilisateur toujours plus fluides et intelligentes.
"""

In [6]:
# 4. Préparer le message pour extraire les personnes
messages = [
    {
        "role": "user",
        "content": f"Extrais toutes les personnes mentionnées dans ce texte: {texte}",
    }
]

In [7]:
# 5. Extraction des personnes sous forme de générateur
extraction_personnes = client.chat.completions.create(
    messages=messages,
    response_model=ExtractionPersonnes,  # Attente d'un objet contenant "personnes"
    max_retries=2,
)

In [8]:
# 7. Affichage du raisonnement et des noms et rôles des personnes extraites

print("Raisonnement du modèle: \n")
print(extraction_personnes.reasoning)


print("\nListe des personnes extraites :\n")
for i, personne in enumerate(extraction_personnes.list_personnes, start=1):
    print(
        f"Personne {i}:\nNom: {personne.nom}, Rôle: {personne.role}, Compagnie: {personne.compagnie}, Age: {personne.age}"
    )
    print("Ou, directement: " + str(personne) + "\n")

Raisonnement du modèle: 

J'ai identifié les noms, âges, rôles et compagnies des personnes mentionnées dans le texte. J'ai ensuite créé un objet JSON pour chaque personne, en respectant le schéma fourni. Les informations ont été extraites directement du texte, et j'ai veillé à ce que les types de données soient corrects (par exemple, l'âge est un nombre entier ou une chaîne de caractères).

Liste des personnes extraites :

Personne 1:
Nom: Thomas, Rôle: ingénieur en intelligence artificielle, Compagnie: NeuroTech Systems, Age: 34
Ou, directement: nom='Thomas' age=34 role='ingénieur en intelligence artificielle' compagnie='NeuroTech Systems'

Personne 2:
Nom: Léa, Rôle: designer UX, Compagnie: VirtuaSoft, Age: 29
Ou, directement: nom='Léa' age=29 role='designer UX' compagnie='VirtuaSoft'

Personne 3:
Nom: Karim, Rôle: chef de projet, Compagnie: Quantum Solutions, Age: 41
Ou, directement: nom='Karim' age=41 role='chef de projet' compagnie='Quantum Solutions'

